In [1]:
import numpy as np
import pandas as pd

Max reward problem

In [2]:
pi = np.array([1, 2])

# A[i,j] = the reward you get when transist from i to j
A = np.array([[3, 2],
              [1, 4]])

B = np.array([[4, 7, 5, 4],
              [6, 1, 5, 3]])

In [3]:
def get_reward(seq, pi, A, B):
    reward = pi[seq[0]] + B[seq[0], 0]
    for i in [1, 2, 3]:
        reward += A[seq[i-1], seq[i]] + B[seq[i], i]

    return reward

In [4]:
seq = [1, 0, 1, 1]
get_reward(seq, pi, A, B)

30

Naive implementation

In [5]:
for x in [0, 1]:
    for y in [0, 1]:
        for z in [0, 1]:
            for w in [0, 1]:
                seq = [x, y, z, w]
                print(x, y, z, w, get_reward(seq, pi, A, B))

0 0 0 0 30
0 0 0 1 28
0 0 1 0 27
0 0 1 1 29
0 1 0 0 21
0 1 0 1 19
0 1 1 0 22
0 1 1 1 24
1 0 0 0 31
1 0 0 1 29
1 0 1 0 28
1 0 1 1 30
1 1 0 0 26
1 1 0 1 24
1 1 1 0 27
1 1 1 1 29


Dynamic programming

In [6]:
alpha = np.zeros((2, 4)) # alpha[i, j] = the maximum reward you can get from START to node[i,j]
backp = np.zeros((2, 4), dtype=int) # to achieve the maximum reward, you should come from node[backp[i,j], j-1]

alpha[:, 0] = pi + B[:, 0]
for i in [1, 2, 3]: # the current col
    for tag_i in range(2): # tag i
        rewards = alpha[:, i-1] + A[:, tag_i] + B[tag_i, i]

        alpha[tag_i, i] = np.max(rewards)
        backp[tag_i, i] = np.argmax(rewards)

max_reward = np.max(alpha[:, 3])
print(max_reward)

last_tag = np.argmax(alpha[:, 3])
seq = [last_tag]
for i in [3, 2, 1]:
    last_tag = backp[last_tag, i]
    seq.append(last_tag)
print(seq[::-1])

31.0
[1, 0, 0, 0]


In [7]:
backp = np.zeros((2, 4), dtype=int)

alpha = pi + B[:, 0]
for i in [1, 2, 3]: # the current col
    rewards = alpha[:, None] + A + B[:, i]
    alpha = np.max(rewards, axis=0)
    backp[:, i] = np.argmax(rewards, axis=0)

max_reward = np.max(alpha)
print(max_reward)

last_tag = np.argmax(alpha)
seq = [last_tag]
for i in [3, 2, 1]:
    last_tag = backp[last_tag, i]
    seq.append(last_tag)
print(seq[::-1])

31
[1, 0, 0, 0]


Put above cells together in a function

In [8]:
def decode(pi, A, B):
    num_tags, num_words = B.shape
    
    alpha = np.zeros((num_tags, num_words))
    backp = np.zeros((num_tags, num_words), dtype=int)
    
    alpha[:, 0] = pi + B[:, 0]
    for i in range(1, num_words): # the current col
        for tag_i in range(num_tags): # tag i
            rewards = alpha[:, i-1] + A[:, tag_i] + B[tag_i, i]

            alpha[tag_i, i] = np.max(rewards)
            backp[tag_i, i] = np.argmax(rewards)
            
    last_tag = np.argmax(alpha[:, -1])
    seq = [last_tag]
    for i in range(num_words-1, 0, -1): # num_words-1, ..., 2, 1
        last_tag = backp[last_tag, i]
        seq.append(last_tag)
    return seq[::-1]

In [9]:
decode(pi, A, B)

[1, 0, 0, 0]

Real-world example: estimate pi, A, B from the treebank corpus

In [10]:
from nltk.corpus import treebank

In [11]:
corpus = treebank.tagged_sents()
print(corpus)

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]


In [12]:
from collections import defaultdict
tag_word_counts = defaultdict(lambda: defaultdict(lambda: 0))
tag_tag_counts  = defaultdict(lambda: 0)

for sent in corpus:
    last_tag = 'START'
    for word, tag in sent:
        tag_word_counts[tag][word.lower()] += 1
        tag_tag_counts[last_tag, tag] += 1
        last_tag = tag

calculate the transition matrix A

In [13]:
tags = []
word_dicts = []
for tag, word_dict in tag_word_counts.items():
    tags.append(tag)
    word_dicts.append(word_dict)

In [14]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
count_matrix = vectorizer.fit_transform(word_dicts)

words = vectorizer.feature_names_
count_matrix = count_matrix.toarray()

In [15]:
df_emission = pd.DataFrame(count_matrix, index=tags, columns=words)

df_emission += 1e-6 # smooth, add a small value to the whole matrix
df_emission = df_emission.div(df_emission.sum(axis=1), axis=0) # normalize, so that every row sums up to 1

In [16]:
df_emission

,!,#,$,%,&,','','30s,'40s,'50s,...,zealand,zenith,zero,zicklin,zinc,zip,zone,zoomed,zuckerman,zurich
``,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,...,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09
VBG,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,...,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10
SYM,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887422e-01,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,...,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07
WDT,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,...,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09
JJ,1.714086e-10,1.714086e-10,1.714086e-10,1.714088e-04,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,...,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10,1.714086e-10
JJS,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,...,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09
:,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,...,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09
LS,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,...,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08
RBR,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,...,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09
VBD,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,...,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10


In [17]:
sent = 'eye drops off shelf'.split() # an example sentence

In [18]:
df_emission[sent] # extract related columns from the full maxtrix, this is the matrix we will use

,eye,drops,off,shelf
``,1.404472e-09,1.404472e-09,1.404472e-09,1.404472e-09
VBG,6.849262e-10,6.849262e-10,6.849262e-10,6.849262e-10
SYM,9.887412e-07,9.887412e-07,9.887412e-07,9.887412e-07
WDT,2.247134e-09,2.247134e-09,2.247134e-09,2.247134e-09
JJ,1.714086e-10,1.714086e-10,3.428175e-04,1.714086e-10
JJS,5.494162e-09,5.494162e-09,5.494162e-09,5.494162e-09
:,1.776163e-09,1.776163e-09,1.776163e-09,1.776163e-09
LS,7.685576e-08,7.685576e-08,7.685576e-08,7.685576e-08
RBR,7.352326e-09,7.352326e-09,7.352326e-09,7.352326e-09
VBD,3.286218e-10,3.286218e-10,3.286218e-10,3.286218e-10


calculate the transition matrix and the pi vector

In [19]:
df_trans = pd.DataFrame(columns=tags, index=tags)
df_pi    = pd.Series(index=tags)

for t1, t2 in tag_tag_counts:
    if t1 == 'START':
        df_pi.loc[t2] = tag_tag_counts[t1, t2]
    else:
        df_trans.loc[t1, t2] = tag_tag_counts[t1, t2]
        
df_pi = df_pi.fillna(0) + 1e-6 # smooth
df_pi = df_pi.div(df_pi.sum()) # normalize

df_trans = df_trans.fillna(0) + 1e-6 # smooth
df_trans = df_trans.div(df_trans.sum(axis=1), axis=0) # normalize

In [20]:
df_pi

``        7.562596e-02
VBG       4.343383e-03
SYM       2.554931e-10
WDT       5.109865e-04
JJ        3.653551e-02
JJS       1.532959e-03
:         2.810424e-03
LS        1.788452e-03
RBR       7.664796e-04
VBD       2.554934e-04
PRP$      7.409300e-03
RBS       5.109865e-04
#         2.554931e-10
''        2.554934e-04
-NONE-    2.095043e-02
VBN       1.788452e-03
DT        2.312213e-01
WP        3.576904e-03
JJR       3.065917e-03
WRB       6.387328e-03
NNS       4.675524e-02
NNP       1.977517e-01
WP$       2.554931e-10
RB        4.471129e-02
UH        2.554934e-04
,         2.554931e-10
-RRB-     2.554931e-10
VB        7.664796e-04
FW        2.554931e-10
$         1.277466e-03
IN        1.290240e-01
VBP       2.554931e-10
-LRB-     1.788452e-03
NNPS      2.554931e-03
CD        8.431273e-03
CC        5.135411e-02
POS       2.554931e-10
PDT       7.664796e-04
VBZ       2.299438e-03
EX        4.343383e-03
TO        1.277466e-03
.         2.554931e-10
NN        4.445580e-02
PRP       6

In [21]:
df_trans

,``,VBG,SYM,WDT,JJ,JJS,:,LS,RBR,VBD,...,POS,PDT,VBZ,EX,TO,.,NN,PRP,RP,MD
``,1.408451e-09,7.042254e-03,1.408451e-09,1.408451e-09,1.112676e-01,1.408451e-09,1.408451e-09,1.408451e-09,1.408451e-09,5.633804e-03,...,1.408451e-09,1.408451e-09,1.830986e-02,2.112676e-02,1.408451e-09,1.408451e-09,9.859154e-02,1.859155e-01,1.408451e-09,1.126761e-02
VBG,3.424658e-03,2.054795e-03,6.849315e-10,6.849315e-10,6.917808e-02,6.849315e-10,2.054795e-03,6.849315e-10,3.424658e-03,1.369864e-03,...,6.849315e-10,6.849322e-04,6.849322e-04,6.849315e-10,4.452055e-02,1.575342e-02,1.465753e-01,1.986301e-02,1.917808e-02,6.849315e-10
SYM,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,...,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07,9.999540e-07
WDT,4.494384e-03,2.247191e-09,2.247191e-09,2.247191e-09,1.123596e-02,2.247193e-03,2.247193e-03,2.247191e-09,2.247191e-09,4.494384e-03,...,2.247191e-09,2.247191e-09,4.494384e-03,2.247193e-03,2.247191e-09,2.247191e-09,6.741575e-03,3.820225e-02,2.247191e-09,2.247193e-03
JJ,2.056908e-03,4.456634e-03,1.714090e-10,1.714090e-10,6.359273e-02,3.428181e-04,2.913953e-03,1.714090e-10,3.428181e-04,8.570451e-04,...,1.714090e-10,1.714090e-10,1.199863e-03,1.714090e-10,1.097017e-02,2.194035e-02,4.475488e-01,5.142271e-04,1.714092e-04,1.714092e-04
JJS,5.494504e-09,1.648352e-02,5.494504e-09,5.494504e-09,1.373626e-01,5.494504e-09,5.494510e-03,5.494504e-09,5.494504e-09,5.494510e-03,...,5.494504e-09,5.494504e-09,5.494510e-03,5.494504e-09,5.494504e-09,2.747253e-02,2.802197e-01,5.494504e-09,5.494504e-09,5.494504e-09
:,4.615384e-02,9.615386e-03,1.923077e-09,3.846155e-03,6.923077e-02,1.923077e-09,1.923077e-09,7.692309e-03,1.923079e-03,2.307692e-02,...,1.923077e-09,1.923077e-09,1.153846e-02,1.923079e-03,1.923079e-03,1.153846e-02,3.846154e-02,3.076923e-02,1.923077e-09,1.538462e-02
LS,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08,1.538457e-01,7.692280e-08,7.692280e-08,7.692280e-08,...,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08,3.846141e-01,7.692280e-08,7.692280e-08,7.692280e-08,7.692280e-08
RBR,7.352939e-09,1.470588e-02,7.352939e-09,7.352939e-09,3.749999e-01,7.352939e-09,1.470588e-02,7.352939e-09,7.352939e-09,7.352946e-03,...,7.352939e-09,7.352939e-09,7.352939e-09,7.352939e-09,7.352946e-03,4.411764e-02,7.352939e-09,7.352939e-09,7.352939e-09,7.352946e-03
VBD,3.614854e-03,1.741702e-02,3.286231e-10,3.286231e-10,4.436411e-02,3.286231e-10,1.971739e-03,3.286231e-10,2.628985e-03,3.286231e-10,...,3.286231e-10,6.572465e-04,3.286231e-10,3.286231e-10,2.398948e-02,6.901085e-03,2.957608e-02,1.150181e-02,1.544528e-02,3.286231e-10


In [22]:
# use the log of probabilities here, so that the 'decode' function works for HMMs
tag_indices = decode(np.log(df_pi.values), np.log(df_trans.values),
                     np.log(df_emission[sent].values))

In [23]:
np.asarray(tags)[tag_indices]

array(['NN', 'NNS', 'IN', 'NN'], dtype='<U6')

In [24]:
sent

['eye', 'drops', 'off', 'shelf']

Explanations of the sentence 'eye drops off shelf' from http://www.vodppl.upm.edu.my/uploads/docs/Analysis%20of%20Ambiguity.pdf#page=2

Eye drops off shelf (Headlines)

Lexical ambiguity: Two possible interpretations - ‘eyedrops’ refers to the cleansing liquid
used to relieve eyes, in this case, the sentence means the items have been taken off the
market; ‘drops’ refer to the verb to fall from certain height; here it means an eye ball has
dropped off a shelf.